In [ ]:
import ee
import datetime
import pandas as pd
import geopandas as gpd
from copy import copy
import json

In [ ]:
ee.Authenticate()
ee.Initialize(project='catchments-climate')

In [ ]:
# Needs shapefiles from https://www.geoplatform.gov/metadata/673067aa-83e8-4e4a-9f02-3f55ed028b72

catchments_gages = gpd.read_file('boundaries-shapefiles-by-aggeco/gages_basins_conus.shp', dtype={'GAGE_ID': 'str'})
catchments_gages.set_crs(epsg=4269, inplace=True)

In [ ]:
with open("STAID_for_drought.json", 'r') as file:
    STAID_for_drought = json.load(file)
    
catchments_gdb = catchments_gages[catchments_gages.GAGE_ID.isin(STAID_for_drought)]

In [ ]:
def make_spatial_aggregation(img, agg_funct=ee.Reducer.mean()):
  """
  Function that spatially aggregates all the variables
  PARAMS:
    img (ee.ImageCollection): image collection containing the variables to be spatially aggregated
  OPTIONAL PARAMS:
    agg_funct (ee.Reducer function): funtion to be used as spatial aggregator. Default: ee.Reducer.mean()
  RETURNS:
    img (ee.ImageCollection): image collection containing the spatially aggregated variables
  """

  img = ee.Image(img)
  date = img.get('system:time_start')
  variables_dict = {"date_agg": date}
  for variable in variables_:
    variable_band = img.select(variable)
    stats = variable_band.reduceRegion(reducer=agg_funct, geometry=roi, scale=30, bestEffort=True)
    variables_dict[variable+"_agg"] = stats.get(variable)
  return img.set(variables_dict)

In [ ]:
def wrapper_roi(dataset,
            variables_,
            start_date,
            end_date,
            roi,
            roi_name,
            model=None):
  print(variables_)
# Load the collection from the Climatic Model
# Filter by date and roi
  if model is not None:
    collection = ee.ImageCollection(dataset) \
              .filter(ee.Filter.eq('model', model)) \
              .filterDate(start_date, end_date) \
              .filterBounds(roi).select(variables_[0])

  else:
    collection = ee.ImageCollection(dataset) \
                .filterDate(start_date, end_date) \
                .filterBounds(roi).select(variables_[0])


  # Sort the filtered collection by date in ascending order
  sorted_collection = collection.sort('system:time_start')

  # Map the function to the image collection
  sorted_collection_changed = sorted_collection.map(make_spatial_aggregation)

  # Extract dates and variables as numpy array
  dates_array = sorted_collection_changed.aggregate_array('date_agg')
  variable_arrays = [sorted_collection_changed.aggregate_array(variable+'_agg') for variable in variables_]

  # Convert the array to a list using getInfo()
  dates_list = dates_array.getInfo()
  variable_lists = [variable_array.getInfo() for variable_array in variable_arrays]

  # Convert dates into strings
  string_timestamps = [datetime.datetime.utcfromtimestamp(int(date_str) // 1000) for date_str in dates_list]

  # Create a DataFrame to store the data
  data = dict(zip(variables_, variable_lists))
  data['date_agg'] = string_timestamps
  df = pd.DataFrame(data, columns=["date_agg"]+variables_)

  # Return DataFrame
  return df

In [ ]:
def make_ee_geometry(gdb, code_name, code_id):
  catchment_gdb = gdb[gdb[code_name]==code_id].iloc[0]
  catchment_geometry_gdb = catchment_gdb.geometry
  if catchment_geometry_gdb.geom_type == 'MultiPolygon':
    catchment_coords = [[list(polygon.exterior.coords)] for polygon in catchment_geometry_gdb.geoms]
  elif catchment_geometry_gdb.geom_type == 'Polygon':
    catchment_coords = [[list(catchment_geometry_gdb.exterior.coords)]]
  else:
    raise ValueError('Geometry is expected to be either a Polygon or a MultiPolygon')
  return ee.Geometry.MultiPolygon(catchment_coords)
  

In [ ]:
# Define start and end dates
start_date = '1980-01-01'
end_date = '2024-10-01'

dataset = 'OREGONSTATE/PRISM/AN81m'
variables = ['tmean', 'ppt']
model = None

code_name="GAGE_ID"
roi_names = catchments_gdb[code_name].values

for j,variable in enumerate(variables):
    for i,roi_name in enumerate(roi_names):
      print(i, roi_name, variable)

      roi = make_ee_geometry(catchments_gdb, code_name, roi_name)
      variables_ = variables[j:j+1]
      df = wrapper_roi(dataset, variables_, start_date, end_date, roi, roi_name, model=model)
      if i==0 and j==0:
        df.rename(columns={variable: str(roi_name)+"|"+variable for variable in variables_}, inplace=True)
        df_all = copy(df)
      else:
        for variable in variables_:
            df_all[str(roi_name)+"|"+variable] = df[variable].values

df_all.to_csv(f'{dataset.replace("/","-")}_{"_".join(variables)}_{start_date}_{end_date}.csv.gz', index=False, compression='gzip')